# MUSIC BRAINZ DATA EXTRACTION

**Purpose** 

We extract the Single/Album  data from MusicBrainz in this file

**Input**
We read the data from ArtistList_July2_withDetails.xlsx and we primarily focus on 2 features from the list of inputs:

1. Artist name
2. Music Brainz Id of the artist extracted manually
    
**Deliverable**    
The output of the file contains the following:

1. Name of the artist
2. Title of the single/album
3. Release date of the single/album
4. Type of the entry whether it's a single/album

In [1]:
# Necessary imports
import pandas as pd
import numpy as np
import json
import urllib
import time

***Import data***

In [2]:
# Read the artists list with handles and load it into a dataframe
xls = pd.ExcelFile('ArtistList_July2_withDetails.xlsx')
master_artists = pd.read_excel(xls, 'FinalArtistListwithHandles_Jul2')

In [7]:
# Create a new dataframe with the following columns
df = pd.DataFrame(columns = ['artist_name','title','release_date','primary_type'])

***Extracting data from MusicBrainz server***

We are extracting MusicBrainz Data using urllib. The url to query is in the following form:
"http://musicbrainz.org/ws/2/release/?query=arid:" + [MusicBrainz ID of the artist] + "&fmt=json&inc=release-groups&limit=100&offset=" + [offset value]

We get the albums and songs list in the form of JSON. The limit=100 in the url means that we get 100 records at a time from the MusicBrainz server. The default is 25 and the maximum we can set is 100, hence we kept limit as 100. 

Offset: It is the offset from the start of the releases list. 
For example: If the artist is Kygo and his musicBrainz ID is ba0e7638-0cd6-4ff4-8987-c3e224d22c23 and has 1000 releases in total:
"http://musicbrainz.org/ws/2/release/?query=arid:ba0e7638-0cd6-4ff4-8987-c3e224d22c23&fmt=json&inc=release-groups&limit=100&offset=0"

The above url returns 100 releases from offset 0, i.e. the first 100 releases.

"http://musicbrainz.org/ws/2/release/?query=arid:ba0e7638-0cd6-4ff4-8987-c3e224d22c23&fmt=json&inc=release-groups&limit=100&offset=100"

Similarly, The above url returns 100 releases from offset 100, i.e. 101-200 releases' information is extracted.

We are leveraging this offset value concept to fetch all the releases information of the artists.

In [8]:
for index, row in master_artists.iterrows():
    # MusicBrainz Server dies down after every 375 requests. Hence We are providing a thread sleep to avoid it.
    time.sleep(5) 
    # Initial offset for an artist's data
    offset = 0
    while True:
        url = "http://musicbrainz.org/ws/2/release/?query=arid:" + row['MusicBrainz']+ "&fmt=json&inc=release-groups&limit=100&offset=" + str(offset)
        contents = urllib.request.urlopen(url).read()
        channels = json.loads(contents)
        
        # Increase the offset by 100 as discussed above
        offset += 100 
        
        # Stopping condition, when there is no release data left
        if len(channels['releases']) == 0:
            break
        for releases in channels['releases']:
            features = []
            
            # Fill the artist column
            features.append(row['Artist'])
            
            # Fill the title column and NaN if not available
            if 'title' in releases:
                features.append(releases['title'])
            else:
                features.append(np.NAN)
                
            # Fill the date column with date of release and NaN if not available
            if 'date' in releases:
                features.append(releases['date'])
            else:
                features.append(np.NAN)
                
            # Fill the release-group column and NaN if not available
            if 'release-group' in releases:
                if 'primary-type' in releases['release-group']:
                    features.append(releases['release-group']['primary-type'])
                else:
                    features.append(np.NAN)
            else:
                features.append(np.NAN)
                
            # Append all the records extracted to the already existing dataframe df    
            df = df.append(pd.Series(features,index = df.columns),ignore_index=True)

***Cleaning musicbrainz data***

In [11]:
# Drop the rows with release dates as null values
df = df.dropna(axis=0, subset=['release_date'])

# Drop the rows with release_date column length 0 i.e. no release date
df = df[df['release_date'].str.len() != 0]

In [18]:
# Convert the release_date column to datetime
df['release_date'] = pd.to_datetime(df['release_date'], format='%Y-%m-%d %H:%M:%S')

In [25]:
# Sort the rows by artist_name, title and release_date columns
df.sort_values(by=['artist_name','title','release_date'],inplace=True)

# Pick the first date of release in case of multiple release dates
df = df.groupby(['artist_name','title']).first()

# Reset index of the dataframe
df.reset_index(inplace=True)
df.head()

,artist_name,title,release_date,primary_type
0,21 Savage,1Night,2018-04-27,Single
1,21 Savage,Bartier Cardi,2017-12-22,Single
2,21 Savage,Enzo,2016-04-24,Single
3,21 Savage,Focus,2019-06-13,Single
4,21 Savage,Free Guwop EP,2015-07-02,Album


***Save the dataframe to file***

In [27]:
df.to_csv('musicbrainz_data.csv')